In [8]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams
using DataStructures
using Clustering
using Distances
using Iterators
using StatsBase
using Training

In [19]:
ee = restore("models/ss/text8_200__i5_v2.model");
dtree,labels = nn_tree(ee)

(NearestNeighbors.BallTree{Float32,Query.AngularDist}
  Number of points: 295775
  Dimensions: 200
  Metric: Query.AngularDist()
  Reordered: true,[("rearrangement",1),("piecemeal",1),("quatre",1),("proteases",1),("fsln",1),("duma",1),("libertad",1),("titration",1),("photosynthesis",1),("qajar",1)  …  ("confined",11),("confined",12),("confined",13),("confined",14),("confined",15),("confined",16),("confined",17),("confined",18),("confined",19),("confined",20)])

In [20]:
@time find_nearest_words(ee,"solvent",dtree,labels )

  8.470255 seconds (13.49 M allocations: 489.062 MB, 27.37% gc time)


20-element Array{Array{Tuple{Tuple{String,Int64},Float32},1},1}:
 [(("solvent",1),0.0f0),(("lp",1),0.38617045f0),(("freight",2),0.4005258f0),(("hanukkah",8),0.4013339f0),(("ancestry",1),0.40245205f0)]                    
 [(("solvent",2),0.00010990189f0),(("used",8),0.36280286f0),(("are",13),0.36325294f0),(("sulfuric",1),0.36509895f0),(("chemistry",3),0.36603138f0)]        
 [(("solvent",3),0.0f0),(("panels",4),0.39183682f0),(("material",2),0.3995582f0),(("lifestyles",1),0.4040893f0),(("examination",12),0.40444997f0)]         
 [(("solvent",4),0.0f0),(("adelaide",17),0.39587548f0),(("viking",3),0.40690184f0),(("rpg",16),0.40706158f0),(("achaemenid",1),0.40727332f0)]              
 [(("solvent",5),0.00010990189f0),(("formats",7),0.3888317f0),(("drilling",1),0.3945168f0),(("phase",12),0.39766484f0),(("tank",16),0.399985f0)]           
 [(("solvent",6),0.0f0),(("reactions",11),0.38621718f0),(("quebecers",1),0.3992329f0),(("participated",20),0.399569f0),(("devolved",1),0.39969772f0)]      

In [7]:
function load_wordsim353(filename="data/corpora/wordsim353/combined.csv")
    wordsims = readdlm(filename, ','; skipstart=1)
    wordpairs = convert(Matrix{String},wordsims[:,1:2])
    groundsim = convert(Vector{Float64},wordsims[:,3])
    (wordpairs, groundsim)
end

load_wordsim353 (generic function with 2 methods)

In [21]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String})
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            wvs_1 = hcat(ee.embedding[wordpairs[ii,1]]...)
            wvs_2 = hcat(ee.embedding[wordpairs[ii,2]]...)
            sims[ii]=1-minimum(pairwise(CosineDist(), wvs_1,wvs_2))
        catch ex
            warn(ex)
            sims[ii]=NaN
        end
    end
    return sims
end
sims = get_sims(ee,map(lowercase,wordpairs))
corspearman(groundsim,sims)

LoadError: LoadError: UndefVarError: wordpairs not defined
while loading In[21], in expression starting on line 15

LoadError: LoadError: UndefVarError: wordpairs not defined
while loading In[22], in expression starting on line 1

In [32]:
function load_SCWS(filename = "data/corpora/SCWS/ratings.txt")
    function get_context2(fulltext)
        Task() do
            skip_next = 0
            for word in split(fulltext, ' ')
                skip_next-=1
                if word == "<b>"
                    skip_next=3 
                end
                skip_next>0 && continue
                produce(lowercase(word))
            end
        end |> collect
    end
    entries = readdlm(filename, '\t', String; quotes=false, comments=false)
    wordpairs = map(lowercase,entries[:,[2,4]])
    groundsim = map(s->parse(Float64,s), entries[:,8])
    contexts = map(get_context2,entries[:,[6,7]])
    (wordpairs,contexts,groundsim,)
end
wordpairs,contexts,groundsim = load_SCWS();
""

""

In [33]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            
            sense1 = Training.WSD(ee, wordpairs[ii,1],contexts[ii,1];skip_oov=true)
            wv1 = ee.embedding[wordpairs[ii,1]][sense1]
            
            sense2 = Training.WSD(ee, wordpairs[ii,2],contexts[ii,2];skip_oov=true)
            wv2 = ee.embedding[wordpairs[ii,2]][sense2]
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end

get_sims (generic function with 2 methods)

In [35]:
local_sims = get_sims(ee,wordpairs,contexts)

2016-06-14T20:07:39.264 - warn: KeyError: aglow not found prefix: "error: "
2016-06-14T20:07:39.765 - warn: KeyError: know-how not found prefix: "error: "
2016-06-14T20:07:42.083 - warn: KeyError: realtor not found prefix: "error: "
2016-06-14T20:07:42.128 - warn: KeyError: bedlam not found prefix: "error: "
2016-06-14T20:07:43.177 - warn: KeyError: riverbank not found prefix: "error: "
2016-06-14T20:07:43.378 - warn: KeyError: tip-off not found prefix: "error: "
2016-06-14T20:07:44.58 - warn: KeyError: exhale not found prefix: "error: "
2016-06-14T20:07:44.603 - warn: KeyError: insufflate not found prefix: "error: "
2016-06-14T20:07:45.388 - warn: KeyError: angstrom not found prefix: "error: "
2016-06-14T20:07:47.369 - warn: KeyError: litigate not found prefix: "error: "
2016-06-14T20:07:47.982 - warn: KeyError: arpeggio not found prefix: "error: "
2016-06-14T20:07:51.006 - warn: KeyError: air-conditioned not found prefix: "error: "
2016-06-14T20:07:51.602 - warn: KeyError: submerge n

2003-element Array{Float64,1}:
  -0.0552722 
  -0.201297  
 NaN         
   0.433096  
   0.287813  
   0.343943  
   0.449379  
  -0.0186976 
   0.25575   
   0.0897515 
   0.0684031 
   0.0637212 
   0.0568659 
   ⋮         
   0.0354389 
   0.252676  
   0.0384336 
  -0.0615323 
   0.134587  
   0.0494938 
  -0.0365748 
   0.00632459
  -0.0790521 
   0.320475  
   0.200819  
   0.0998082 

In [36]:
corspearman(groundsim,local_sims)

0.4036875361072294